In [47]:
#pip install selenium

In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbs


In [49]:
"""
TO DO:
- Entrar a la página de la facultad de ingenieria y probar la clave 1413 (Introducción a la Economia) y extraer la tabla de datos
- Colocar los datos en un excel y guardarlo
- Generalizar la función para cualquier clabe
- Anidar los datos extraidos en un único excell


from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

#Accede al navegador y a la página de la facultad
driver = webdriver.Chrome()
driver.get("https://www.ssa.ingenieria.unam.mx/horarios.html")

campo_clave = driver.find_element("id","clave")
buscar = driver.find_element("id","buscar")
campo_clave.send_keys("1413")
buscar.click()

import re
import requests
from colorama import Fore

website = "https://www.ssa.ingenieria.unam.mx/horarios.html"
resultado = requests.get(website)
content = resultado.text
print(content)

time.sleep(5)

driver.close()
"""


'\nTO DO:\n- Entrar a la página de la facultad de ingenieria y probar la clave 1413 (Introducción a la Economia) y extraer la tabla de datos\n- Colocar los datos en un excel y guardarlo\n- Generalizar la función para cualquier clabe\n- Anidar los datos extraidos en un único excell\n\n\nfrom selenium import webdriver\nfrom selenium.webdriver.common.keys import Keys\nimport time\n\n#Accede al navegador y a la página de la facultad\ndriver = webdriver.Chrome()\ndriver.get("https://www.ssa.ingenieria.unam.mx/horarios.html")\n\ncampo_clave = driver.find_element("id","clave")\nbuscar = driver.find_element("id","buscar")\ncampo_clave.send_keys("1413")\nbuscar.click()\n\nimport re\nimport requests\nfrom colorama import Fore\n\nwebsite = "https://www.ssa.ingenieria.unam.mx/horarios.html"\nresultado = requests.get(website)\ncontent = resultado.text\nprint(content)\n\ntime.sleep(5)\n\ndriver.close()\n'

In [50]:
df = pd.read_excel('Excel/Campeones.xlsx')
df = df.sort_values('Clave')
df = df.reset_index(drop=True)
lista_horarios = list()
claves_excluidas = []
df

,Clave,Gpo,Profesor,Tipo,Horario,Días,Salón,Nombre,Cupo,Puntaje
0,840,5,M.C. LAURA SANDOVAL MONTAÑO,T,11:00 a 13:00,"Mar, Jue",B304,Sistemas Operativos,3,NaN
1,1000,4,M.A Feher Guillermina A,ENALLT,13:00 a 15:00,"Lun, Mar, Jue",A204,English,0,NaN
2,1562,1,M.C. EDGAR BALDEMAR AGUADO CRUZ,T,17:30 a 19:00,"Lun, Mie",A306,Circuitos Electricos,0,NaN
3,1643,4,M.I. TANYA ITZEL ARTEAGA RICCI,T,17:00 a 19:00,"Mar, Jue",B204,Admin. Proy. Softw.,5,NaN
4,1644,8,M.A. FRANCISCO JAVIER ROJAS DURAN,T,07:00 a 10:00,"Vie, Sab",B103,Bases de Datos,1,NaN
5,1644,1,ING. FERNANDO ARREOLA FRANCO,T,07:00 a 09:00,"Lun, Mie, Vie",A301,Bases de Datos,2,NaN
6,1644,3,ING. LUCILA PATRICIA ARELLANO MENDOZA,T,11:00 a 13:00,"Lun, Mie, Vie",B103,Bases de Datos,0,NaN
7,1645,5,M. I. MARIA DEL SOCORRO GUEVARA RODRIGUEZ,T,15:00 a 17:00,"Mar, Jue",J110,Dis. Dig. Mod,8,NaN
8,6562,8,M.I. PATRICIA HONG CIRION,L+,20:00 a 22:00,Mar,P215,Lab. Circuitos Electricos,5,NaN
9,6562,7,M.I. JORGE ALBERTO UC MARTIN,L+,18:00 a 20:00,Mar,P215,Lab. Circuitos Electricos,4,NaN


In [51]:
for clave in claves_excluidas:
   df.drop(df[df['Clave']==clave].index, inplace=True)
materias = len(df.Clave.unique())
df.Clave.unique()



array([ 840, 1000, 1562, 1643, 1644, 1645, 6562, 6644, 6645], dtype=int64)

In [52]:
#Verificación por horario
def haySolapeHoras(a,b): #DataFrame, comparativa a comparativa b
    if b.Inicio_min - a.Inicio_min == 0: #empiezan a la misma hora
        return True
    if b.Inicio_min - a.Inicio_min > 0: #b empieza después que a
        if(b.Inicio_min - a.Inicio_min < a.Duracion*60):
            return True
    else:
        if(abs(b.Inicio_min - a.Inicio_min) < b.Duracion*60):
            return True 
    return False
    

In [53]:
def noHaySolape(horario_actual,materia):    
    """
    Verifica si existe un solape entre un potencial horario_actual y una materia (renglón del dataframe)

    Parameters
    ----------
        horario_actual: dataFrame
            Es el horario_actual que se va armando
        materia: series
            Es la fila que contiene toda la info de la materia que se quiere meter
    """
    orden = ['Lun', 'Mar', 'Mie', 'Jue', 'Vie', 'Sab']
    for i in range(0,len(horario_actual)):
        prod = horario_actual.iloc[i].loc[orden] * materia.loc[orden]

        #Ejecuta la funcion all para ver si todos los elementos son cero
        if(not (prod == 0).all()):
            if(haySolapeHoras(horario_actual.iloc[i],materia)):
                return False
    return True

# Get dummies de los días

In [54]:
#Crea el encabezado de los dummy values de los días (en orden)
orden = ['Lun', 'Mar', 'Mie', 'Jue', 'Vie', 'Sab'] 

#Obtiene los dummy values directo del df
dummy_dias = pd.Series(df.Días).str.get_dummies(sep=', ')

# Reordena y añade los valores de las variables ficticias a la serie "dias" y cambia los NaN por 0
dummy_dias = dummy_dias.reindex(orden, axis=1).fillna(0, downcast='int64')

#Añade este nuevo df al df original
df = df.assign(**dummy_dias)

# Get dummies de las horas

In [55]:
#Crea una series de las horas que se imparte esa materia
horas_sep = df.Horario.str.split(" a ")

#Crea un diccionario con la series anterior, la clave es el índice de la serie y el valor la tupla de horas
diccionario = dict(zip(horas_sep.index, horas_sep.values))

#A partir de este diccionario, crea un dataframe llamado horas y nombra sus columnas
horas = pd.DataFrame.from_dict(diccionario).transpose()
horas.columns = ['Inicio','Fin']

#Une las horas al dataframe original
df = df.assign(**horas)


In [56]:
#Genera una series que contiene las horas y los minutos como tupla
hrs_y_min = df.Inicio.str.split(":")

#Obtiene la hora de inicio en terminos de minutos y crea una nueva columna en el df
minutos = hrs_y_min.str[0].astype(int)*60 +  hrs_y_min.str[1].astype(int)

#Une la variable minutos como inicio
df = df.assign(Inicio_min = minutos)

#obtiene la hora final en terminos de minutos y crea una nueva columna en el df
hrs_y_min = df.Fin.str.split(":")
minutos = hrs_y_min.str[0].astype(int)*60 +  hrs_y_min.str[1].astype(int)

#Une la variable miniutos como fin
df = df.assign(Fin_min = minutos)



In [57]:

#Calcula la duración de cada clase en horas
duracion = df.Fin_min - df.Inicio_min
df = df.assign(Duracion = duracion/60)
df

,Clave,Gpo,Profesor,Tipo,Horario,Días,Salón,Nombre,Cupo,Puntaje,...,Mar,Mie,Jue,Vie,Sab,Inicio,Fin,Inicio_min,Fin_min,Duracion
0,840,5,M.C. LAURA SANDOVAL MONTAÑO,T,11:00 a 13:00,"Mar, Jue",B304,Sistemas Operativos,3,NaN,...,1,0,1,0,0,11:00,13:00,660,780,2.0
1,1000,4,M.A Feher Guillermina A,ENALLT,13:00 a 15:00,"Lun, Mar, Jue",A204,English,0,NaN,...,1,0,1,0,0,13:00,15:00,780,900,2.0
2,1562,1,M.C. EDGAR BALDEMAR AGUADO CRUZ,T,17:30 a 19:00,"Lun, Mie",A306,Circuitos Electricos,0,NaN,...,0,1,0,0,0,17:30,19:00,1050,1140,1.5
3,1643,4,M.I. TANYA ITZEL ARTEAGA RICCI,T,17:00 a 19:00,"Mar, Jue",B204,Admin. Proy. Softw.,5,NaN,...,1,0,1,0,0,17:00,19:00,1020,1140,2.0
4,1644,8,M.A. FRANCISCO JAVIER ROJAS DURAN,T,07:00 a 10:00,"Vie, Sab",B103,Bases de Datos,1,NaN,...,0,0,0,1,1,07:00,10:00,420,600,3.0
5,1644,1,ING. FERNANDO ARREOLA FRANCO,T,07:00 a 09:00,"Lun, Mie, Vie",A301,Bases de Datos,2,NaN,...,0,1,0,1,0,07:00,09:00,420,540,2.0
6,1644,3,ING. LUCILA PATRICIA ARELLANO MENDOZA,T,11:00 a 13:00,"Lun, Mie, Vie",B103,Bases de Datos,0,NaN,...,0,1,0,1,0,11:00,13:00,660,780,2.0
7,1645,5,M. I. MARIA DEL SOCORRO GUEVARA RODRIGUEZ,T,15:00 a 17:00,"Mar, Jue",J110,Dis. Dig. Mod,8,NaN,...,1,0,1,0,0,15:00,17:00,900,1020,2.0
8,6562,8,M.I. PATRICIA HONG CIRION,L+,20:00 a 22:00,Mar,P215,Lab. Circuitos Electricos,5,NaN,...,1,0,0,0,0,20:00,22:00,1200,1320,2.0
9,6562,7,M.I. JORGE ALBERTO UC MARTIN,L+,18:00 a 20:00,Mar,P215,Lab. Circuitos Electricos,4,NaN,...,1,0,0,0,0,18:00,20:00,1080,1200,2.0


In [58]:
"""TO DO:
- Generar un código que se ejecute en paralelo 
- Generar dos ramas de horarios en paralelo 
- Extender hasta la mayor cantidad de ramas posibles

"""

'TO DO:\n- Generar un código que se ejecute en paralelo \n- Generar dos ramas de horarios en paralelo \n- Extender hasta la mayor cantidad de ramas posibles\n\n'

In [59]:
def combinarMaterias(df,horario,renglon_actual):
    """
    Es una función recursiva que toma los parámetros para ir concatenando un horario

    Parameters
    ------------
        df: dataFrame
            El dataframe que contiene todas las materias
        horario: dataFrame
            El horario que se va formando
        renglon_actual: int
            El indice del renglón del df que se concatena
    
    """
    if renglon_actual < len(df):
        #Concatena el horario con la fila con el índice 'renglon_actual' 
        new_row = df.copy()[df.index == renglon_actual]
        horario = pd.concat([horario, new_row])
        siguiente = renglon_actual + 1
        #El programa va a corroborar si las siguientes filas son compatibles
        for i in range(siguiente,len(df)):
            clv_materia_sig = df.iloc[i]['Clave']
            if ((horario['Clave']==clv_materia_sig).any()):
                """
                TODO:
                Implememtar la verificación con materias asimétricas
                
                """
                continue
            elif noHaySolape(horario,df.iloc[i]):
                combinarMaterias(df,horario,i) #Al ser recursiva, va concatenando las filas del excel que no se solapen y sean diferentes materias
        print("De regreso")
        
    if(len(horario.Clave.unique()) == materias): #Si ya se tienen todas las materias 
        lista_horarios.append(horario)
        #plotHorario(horario,len(lista_horarios))
       

In [60]:
def generarHorarios():
    """
    Genera los horarios, realiza un arbol de posibilidades por cada opcion de la primer materia
    """
    n=0
    #Realiza este proceso mientras que se trate de la misma materia, el dataframe debe estar ordenado por clave
    
    while(df.iloc[n].Clave == df.Clave.unique()[0]): 
        horario = pd.DataFrame(columns = df.columns) #crea un df con las mismas columnas para ir armando el horario
        combinarMaterias(df,horario,n)
        n = n+1

In [61]:
def includes(horario,materia):
    for i in range(0,len(horario)): #Checa con todas las materias del horario
        if (materia.Gpo == horario.iloc[i].Gpo and materia.Clave == horario.iloc[i].Clave):
            return True
    return False

In [62]:
def hayVinculoAtras(df,n):
    if(n>1):
        if (df.iloc[n].Gpo == df.iloc[n-1].Gpo and df.iloc[n].Clave == df.iloc[n-1].Clave):
            return True
    return False

In [63]:
def isClaveRepetida(horario,materia):
    for i in range(0,len(horario)): #Checa con todas las materias del horario
        if (materia.Clave == horario.iloc[i].Clave):
            return True
    return False

In [64]:
def isOpen_link(horario,materia):
    """
    Cambia la bandera open_link si la materia está vinculada con alguna del horario
    Una materia está vinculada con el horario si tiene misma clave y grupo.

    Parameters
    ---------
        horario: DataFrame
            Es el horario que se está armando
        materia: Series
            Es una materia con la que se está comprobando
    """
    global open_link
    for i in range(0,len(horario)): #Checa con todas las materias
        if (materia.Gpo == horario.iloc[i].Gpo and materia.Clave == horario.iloc[i].Clave):
            open_link = True #Se vinculan únicamente si coincide alguna materia en grupo y clave
            return True
    return False
    

In [65]:
def plotMateria(df,gnt):
    import random
    R = random.uniform(0.3,0.75)
    G = random.uniform(0, 0.1)
    B = random.uniform(0.6, 1)
    
    i = 0
    hora = (df.Inicio_min/60)
    duracion = df.Duracion
    
    for cat in df.iloc[9:15]: #Lo que hace es particionar el df a los días de la semana
        if cat == 1:
            gnt.broken_barh([(i,1)], (hora, duracion) ,facecolors = (R, G, B))
            nombre_materia = df.Nombre[:10]                                                     #Corta el texto para que quepa en el cuadro
            nombre_profe = df.Profesor.split(" ")
            nombre_pila = nombre_profe[1]
            nombre_pila = nombre_pila[:5]
            apellido = nombre_profe[-2]
            nombre_pila = df.Profesor[:7]
            inicial = apellido [:1]
            nombre = nombre_materia + "\n" + str(df.Gpo) + " " + nombre_pila  + " " + inicial                                    
            gnt.text(i+0.1,hora+1.5,nombre,color="white",fontweight = "bold",fontsize = "small") #x,y,texto,color,grosor y tamaño
        i = i+1


In [66]:
def plotHorario(horario, numero):
    import matplotlib.pyplot as plt
    fig, gnt = plt.subplots()
    gnt.set_ylim(6,23 )
    gnt.set_xlim(0,6)
    gnt.set_xlabel('Día')
    gnt.set_ylabel('Horas')

    #horas = ['7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22']
    dias = ['Lunes','Martes','Miércoles','Jueves','Viernes','Sábado']
    plt.gca().invert_yaxis()
    gnt.set_yticks(np.arange(7, 23,1))
    gnt.set_xticks(np.arange(0,6,1))
    gnt.set_xticklabels(dias,ha="left")
    gnt.grid(True)

    for i in range(0,len(horario)):
        plotMateria(horario.iloc[i],gnt)

    plt.savefig("Horarios/opción" + str(numero) + ".png")
    plt.close()

In [67]:
def clearCarpeta():
    import os, shutil
    folder = 'Horarios/'
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('No fue posible borrar la carpeta %s. Excepción: %s' % (file_path, e))
    

In [68]:
def imprimirHorarios():
    clearCarpeta()
    if len(lista_horarios) == 0:
        print("No se puede generar ningún horario con las materias ingresadas. POSIBLE SOLAPE")
        return 0
    n=1
    for horario in lista_horarios:
        plotHorario(horario,n)
        #print(horario.iloc[:,0:8].drop(['Tipo','Cupo'],axis=1))
        horario.drop(horario.tail(1).index,inplace=True)
        n+=1
    

In [69]:
#Crea un df vacío con las mismas columnas que el df del excel
horario = pd.DataFrame(columns = df.columns)

#cero = pd.Series([0,0,0,0,0,0], index = orden).astype(object)
generarHorarios()
print("Horarios generados: " + str(len(lista_horarios)))
imprimirHorarios()


***************
0    Sistemas Operativos | M.C. LAURA SANDOVAL MONTAÑO
dtype: object
***************
0    Sistemas Operativos | M.C. LAURA SANDOVAL MONTAÑO
1                    English | M.A Feher Guillermina A
dtype: object
***************
0    Sistemas Operativos | M.C. LAURA SANDOVAL MONTAÑO
1                    English | M.A Feher Guillermina A
2    Circuitos Electricos | M.C. EDGAR BALDEMAR AGU...
dtype: object
***************
0    Sistemas Operativos | M.C. LAURA SANDOVAL MONTAÑO
1                    English | M.A Feher Guillermina A
2    Circuitos Electricos | M.C. EDGAR BALDEMAR AGU...
3    Admin. Proy. Softw. | M.I. TANYA ITZEL ARTEAGA...
dtype: object
***************
0    Sistemas Operativos | M.C. LAURA SANDOVAL MONTAÑO
1                    English | M.A Feher Guillermina A
2    Circuitos Electricos | M.C. EDGAR BALDEMAR AGU...
3    Admin. Proy. Softw. | M.I. TANYA ITZEL ARTEAGA...
4    Bases de Datos | M.A. FRANCISCO JAVIER ROJAS D...
dtype: object
***************
0    Sist